In [4]:
import pandas as pd
import numpy as np
import joblib 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

print("Ferramentas de Modelagem importadas.")

Ferramentas de Modelagem importadas.


In [ ]:
import pandas as pd
import os # Necessário para o diagnóstico de caminho

# Carregamento do dataset limpo
CSV_FILE = '../../data/students_clean.csv'

# Diagnóstico de caminho (DEBUG)
# os.path.abspath(CSV_FILE) transforma o caminho relativo em um caminho completo
print(f"DEBUG: Tentando carregar de: {os.path.abspath(CSV_FILE)}")

try:
    df = pd.read_csv(CSV_FILE)
    
    # Define as features (X) e o target (y)
    X = df.drop(columns=['Pontuacao_Prova_Final', 'Aprovado']) 
    y = df['Pontuacao_Prova_Final'] # Target para Regressão

    print("Dataset limpo carregado com sucesso.")
    print(f"Features (X): {X.shape}, Target (y): {y.shape}")
    
except FileNotFoundError:
    print(f"ERRO: Arquivo {CSV_FILE} não encontrado. Verifique o caminho absoluto impresso acima.")
    X, y = None, None # Define X e y como None em caso de erro para evitar NameError na próxima célula
except Exception as e:
    print(f"Ocorreu um erro ao carregar os dados: {e}")

DEBUG: Tentando carregar de: c:\Users\guili\Projeto-Machine-learning\notebooks\data\students_clean.csv
ERRO: Arquivo ../data/students_clean.csv não encontrado. Verifique o caminho absoluto impresso acima.


In [ ]:



X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)


X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Dados divididos em Treino, Validação e Teste (60/20/20).")
print(f"Treino: {X_train.shape[0]} amostras")
print(f"Validação: {X_val.shape[0]} amostras")
print(f"Teste: {X_test.shape[0]} amostras")

NameError: name 'train_test_split' is not defined

In [ ]:

model = LinearRegression()
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)


feature_importance = pd.Series(model.coef_, index=X_train.columns).sort_values(ascending=False)
print("Coeficientes do Modelo (Importância das Features):")
print(feature_importance)


joblib.dump(model, '../models/baseline_reg.pkl')
print("\nModelo Baseline salvo em ../models/baseline_reg.pkl")